# Input files

DEAGO requires the following for all analyses:

* [Count data directory](#Count-data-directory) (*[example](deago_tutorial/counts)*)   
 *directory containing count files (one per sample)*
 
  

* [Targets file](#Targets-file) (*[example](deago_tutorial/targets.txt)*)     
  _sample (filename) to experimental condition mappings_  

If you want to perform GO enrichment analyses or show gene symbols, you will also need:

* [Annotation file](#Annotation-file) (*[example](deago_tutorial/ensembl_mm10_deago_formatted.tsv)*)      
  _required for gene name annotation and GO term enrichment analysis_ 
  
For bespoke analyses you will also need to pre-generate a config file and use this as an input for some of the component scripts.  

* [Configuration file](#Configuration-file) (*[example](lrt_example/deago.config)*)  
  _key/value parameters required for repeated or bespoke analyses_ 

***

## Count data directory

DEAGO looks in a single folder for all of your count data, one file per sample. You will need to tell DEAGO the location or path for this folder.

To work with different types of expression count files, DEAGO uses several command line options:

| Command line option | Description                                          |
| :------------------ | :--------------------------------------------------- |
| --count_type        | type of count file [_expression_ or _featurecounts_] |
| --count_column      | number of column containing count values             |
| --skip_lines        | number of lines to skip in count file                |
| --count_delim       | count file delimiter                                 |
| --gene_ids          | name of column containing gene identifiers           |

You don't always need to use these options. By default, DEAGO assumes the following:

* `--count_column 5`
* `--skip_lines 0`
* `--gene_ids 'GeneID'`
* `--count_delim ","`

These are the parameters for the expression count files which are produced for all organisms processed by the Sanger pathogens RNA-Seq expression pipeline.

And example of a default expression count file would be:

    "Seq ID",GeneID,"Locus Tag","Feature Type","Total Reads Mapping","Total RPKM (Reads Mapped)",
    "Total Reads Mapping (Reads Mapped to Gene Models)","Total RPKM (Reads Mapped to Gene Models)",
    "Sense Reads Mapping","Sense RPKM (Reads Mapped)","Sense RPKM (Reads Mapped to Gene Models)",
    "Antisense Reads Mapping","Antisense RPKM (Reads Mapped)","Antisense RPKM (Reads Mapped to Gene Models)"
    1,ENSG00000266468,,,0,0,0,0,0,,,0,,
    1,ENSG00000228682,,,0,0,0,0,0,,,0,,
    1,ENSG00000202027,,,0,0,0,0,0,,,0,,
    1,ENSG00000235777,,,0,0,0,0,0,,,0,,
    1,ENSG00000230718,,,0,0,0,0,0,,,0,,

Gene identifiers are in the second column `GeneID` (e.g. ENSG00000266468) and the gene counts are in the fifth column (`column 5`). The first line is the header (i.e. no lines to skip) and the fields are comma-delimited (,).

There is also a preset for featureCounts files `--count_type featurecounts` which assumes:
	
* `--count_column 7`
* `--skip_lines 1`
* `--gene_ids 'Geneid'`
* `--count_delim "\t"`											

Gene identifiers are in the first column `Geneid` (e.g. ENSMUSG00000090025) and the gene counts are in the last column (`column 7`). The first line is a comment which gives the details of the program and command used to generate the count file and the fields are tab-delimited (\t).

    # Program:featureCounts v1.4.5-p1; Command:"featureCounts" "-O" "-T" "1" "-t" "exon" "-g" "gene_id" "-a"
    "/lustre/scratch118/infgen/pathogen/pathpipe/refs/Mus/musculus/Mus_musculus_mm10.gtf" "-o"
    "390176.pe.markdup.bam.featurecounts.csv" "390176.pe.markdup.bam"
    Geneid  Chr     Start   End     Strand  Length  390176.pe.markdup.bam
    ENSMUSG00000090025      1       3054233 3054733 +       501     0
    ENSMUSG00000064842      1       3102016 3102125 +       110     0

You can also use other count file formats with DEAGO as long as you configure the options in the table above accordingly.

***

## Targets file

DEAGO also requires a targets file which tells it the counts file associated with each sample and the experimental conditions which were applied.

Each row corresponds to a sample and there are three required columns:

* `filename`
  _name of the sample count file in the counts directory_
* `condition`
  _experimental condition that was applied_
* `replicate`
  _number or phrase representing a replicate group_

Let's take a look at an example targets file:

Notice that in addition to the three expected columns we also have `cell_type` and `treatment`. That's because this example dataset had two factors, cell type (`WT`/`KO`) and treatment (`Ctrl`/`IL22`). 

Currently, DEAGO can only perform single factor analyses. So, we concatenate the cell type and treatment values in the `condition` column (e.g. `WT_Ctrl`). See [Bespoke analyses](Bespoke-analyses.ipynb) for how to adapt the DEAGO output for multi-factor analyses.

In this dataset we also had 4 biological replicates and 2 technical replicates for each condition. This is represented in the `replicate` column e.g. replicate `1.2` is biological replicate 1 and technical replicate 2.

***

## Annotation file

An annotation file is required if you want to include gene symbols in the results tables or if you want to perform GO term enrichment analyses with DEAGO. The annotation file maps the gene identifiers with gene symbols and/or GO terms.

Let's take a look at an example of a DEAGO-formatted annotation file:

There is one row per gene.  However, not all genes need to be accounted for in the annotation file. The annotation file must contain at least 2 columns. The first column should always contain the gene identifiers (`Geneid` column in featurecounts files). The second and/or third column should contain the gene symbols or GO terms associated with the gene. If there are multiple genes or GO terms associated with a gene they should be concatenated (joined together) with semi-colons (e.g. GO:0005634;GO:0006355;GO:0031519).

For more information on preparing an annotation file for DEAGO (i.e. from Ensembl BioMart output) see [Preparing an annotation file](Preparing-an-annotation-file.ipynb).

***

## Configuration file

DEAGO uses a config file containing tab-delimited key/value pairs to define the parameters for each analysis. 

For a featurecounts QC it will look something like this:

We can ask DEAGO to generate this file using the `--build_config` options:

In [ ]:
    deago --build_config -c deago_tutorial/counts -t deago_tutorial/targets.txt

This will generate a file called `deago.config` as part of the analysis which is built using the command line parameters you provide and will look similar to the content above.

We can also use an existing config file if we want to re-run an analysis:

In [ ]:
    deago --config deago.config

DEAGO can also be run in stages (e.g. `build_deago_config`, `build_deago_markdown`, `deago_markdown_to_html`) if you want to do a bespoke analysis. See [Bespoke analyses](Bespoke-analyses.ipynb) for more information.